**1) Importar librerías**

In [3]:
import re, time, os, pathlib
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

**2) Configuración**

In [4]:
# ---------- CONFIG GLOBAL ----------
OUTPUT_DIR = pathlib.Path("ls_dr10_cutouts_combined")
FITS_DIR   = OUTPUT_DIR / "fits"            # carpeta única para todos los FITS
OUTPUT_CSV = "lsdr10_paths.csv"

LAYER     = "ls-dr10"   # capa DR10
PIXSCALE  = 0.262       # arcsec/pix
SIZE      = 512         # lado (pix)
TIMEOUT_S = 60
WORKERS   = 8
SLEEP_S   = 0.05
BANDS     = "griz"      # DR10 soporta g,r,i,z

FITS_DIR.mkdir(parents=True, exist_ok=True)

**3) Funciones helper**

In [5]:
# --- para SDSS-DR14 (columna 'anillos') ---
ANILLOS_LABEL = {
    0:  "none",
    4:  "inner",
    8:  "outer",
    12: "inner+outer",
}

def to_int(x, default=0):
    """Convierte un valor a entero.

    Args:
        x (float | str | None): Valor a convertir.
        default (int, optional): Valor por defecto en caso de error. Defaults to 0.

    Returns:
        int: Valor convertido o el valor por defecto.
    """
    try:
        return int(x)
    except Exception:
        return default

def ring_type_from_code(code: int) -> str:
    """Convierte un código de anillo a su etiqueta correspondiente.

    Args:
        code (int): Código del anillo.

    Returns:
        str: Etiqueta del anillo o "unknown" si el código no es válido.
    """
    return ANILLOS_LABEL.get(code, "unknown")

# --- cutout URL ---
def fits_url(ra, dec, size=SIZE, bands=BANDS, layer=LAYER, pixscale=PIXSCALE):
    return (f"https://www.legacysurvey.org/viewer/fits-cutout"
            f"?ra={ra:.8f}&dec={dec:.8f}&layer={layer}"
            f"&pixscale={pixscale:.6f}&size={size}&bands={bands}")

**4) SDSS-DR14 (Descarga de FITS, bandas griz)**

In [7]:
def fetch_one_sdss(row, outdir: pathlib.Path):
    """Descarga un FITS de SDSS basado en la información de una fila del DataFrame.
    
    Args:
        row (dict): Fila del DataFrame con las columnas 'ra', 'dec', 'objID', 'anillos'.
        outdir (pathlib.Path): Directorio donde se guardará el archivo FITS.

    Returns:
        dict: Diccionario con las claves 'fits_url', 'fits_path', 'status'.
    """
    # row debe tener: ra, dec, objID, anillos
    if pd.isna(row["ra"]) or pd.isna(row["dec"]):
        return {"fits_url": None, "fits_path": None, "status": "skip_nan"}
    ra   = float(row["ra"])
    dec  = float(row["dec"])
    oid  = str(row.get("objID", f"row{row.get('_idx', 0)}"))
    code = to_int(row.get("anillos", 0))
    rtype = ring_type_from_code(code)

    # Nombre de archivo fits a partir de RA, DEC, objID, tipo de anillo
    base = f"SDSS_{oid}_{rtype}_ra{ra:.6f}_dec{dec:.6f}_s{SIZE}"
    dest = outdir / f"{base}.fits"
    url  = fits_url(ra, dec)

    status = "ok"

    try:
        # Descargar si no existe o está vacío
        if not dest.exists() or dest.stat().st_size == 0:
            r = requests.get(url, timeout=TIMEOUT_S)
            if r.ok and r.content:  # Si la respuesta es válida, guardar el contenido
                dest.write_bytes(r.content)
            else:   # Respuesta no válida, registrar el estado del error
                status = f"http_{r.status_code}"
    except Exception as e:
        status = f"error:{e.__class__.__name__}"

    time.sleep(SLEEP_S)
    return {"fits_url": url, "fits_path": str(dest), "status": status}


# ---------------- PRINCIPAL ----------------
sdss_out_df = None

df_sdss = pd.read_csv("dataset.csv")

# Incluir solo filas con datos de ANILLOS_LABEL
df_sdss = df_sdss[df_sdss["anillos"].isin(ANILLOS_LABEL.keys())].copy()
# imprimir el conteo por tipo de anillo
print(f"[SDSS] Conteo por tipo de anillo:\n{df_sdss['anillos'].map(ANILLOS_LABEL).value_counts()}")

for col in ["ra", "dec", "anillos"]:
    if col not in df_sdss.columns:
        raise SystemExit(f"[SDSS] Falta la columna '{col}'. Presentes: {list(df_sdss.columns)[:20]}")

# columnas derivadas
df_sdss["anillos"]    = df_sdss["anillos"].apply(to_int)
df_sdss["ring_type"]  = df_sdss["anillos"].apply(ring_type_from_code)
df_sdss["is_partial"] = (df_sdss["anillos"] == 16)
df_sdss["_idx"] = range(len(df_sdss))

# descargas
rows = df_sdss.to_dict(orient="records")
downloads = []
with ThreadPoolExecutor(max_workers=WORKERS) as ex:
    futs = [ex.submit(fetch_one_sdss, row, FITS_DIR) for row in rows]
    for f in as_completed(futs):
        downloads.append(f.result())

dl_df = pd.DataFrame(downloads)

assert len(dl_df) == len(df_sdss), "[SDSS] #descargas != #filas"

sdss_out_df = pd.concat([df_sdss.reset_index(drop=True),
                        pd.DataFrame(dl_df, columns=["fits_url","fits_path","status"])], axis=1)
sdss_out_df["source"] = "sdss_anillos"

sdss_out_df.to_csv(OUTPUT_DIR / OUTPUT_CSV, index=False)
print(f"[SDSS] Filas: {len(sdss_out_df)}, OK: {(sdss_out_df.status=='ok').sum()}")

ok = (sdss_out_df.status == "ok").sum()
print(f"Guardado CSV: {OUTPUT_DIR / OUTPUT_CSV}")
print(f"Imágenes en: {FITS_DIR}")
print(f"Descargas OK: {ok} / {len(sdss_out_df)}")
print("\nConteo por ring_type:")
print(sdss_out_df["ring_type"].value_counts(dropna=False))

[SDSS] Conteo por tipo de anillo:
anillos
none           6660
inner           857
inner+outer     372
outer           186
Name: count, dtype: int64
[SDSS] Filas: 8075, OK: 7999
Guardado CSV: ls_dr10_cutouts_combined\lsdr10_paths.csv
Imágenes en: ls_dr10_cutouts_combined\fits
Descargas OK: 7999 / 8075

Conteo por ring_type:
ring_type
none           6660
inner           857
inner+outer     372
outer           186
Name: count, dtype: int64
